In [1]:
import pandas as pd

# Read CSV file, including the first row
data = pd.read_csv('board_evals.csv', header=None)

# Define the column names as a list
column_names = ['Score', 'Captured', 'Potential', 'Regular Pawns', 'Kings', 'Capturables', 'Semi Capturables', 'Uncapturables', 'At Middle', 'Far', 'Is Over', 'Outcome']
data.columns = column_names

In [2]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Scale the first column values
scaler = StandardScaler()
data['Score'] = scaler.fit_transform(np.array(data[['Score']]))
print(data.head(10))

      Score  Captured  Potential  Regular Pawns  Kings  Capturables  \
0 -1.606480         0          1              0      0            0   
1 -1.547084         0          0              0      0            0   
2 -1.665875         0          1              0      0            0   
3 -1.844062         0         -1              0      0            0   
4 -1.309502         0          2              0      0            0   
5 -1.428293         0          7              0      0            0   
6 -0.715548         1          8              1      0           -1   
7 -1.190711         1          0              0      0            0   
8 -1.250107         1         -2              0      0            0   
9 -1.190711         1          4              0      0            0   

   Semi Capturables  Uncapturables  At Middle  Far  Is Over  Outcome  
0                 1             -1          1    0        0       -1  
1                 0              1          0    0        0        1  
2    

In [3]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and the target (y)
X = np.array(data.drop(columns=['Outcome']))  # Features (all columns except 'Outcome')
y = np.array(data['Outcome'])  # Target variable ('Outcome' column)

# Split the data into training (70%) and test (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
import torch
import torch.nn as nn

input_size = X_train.shape[1]
hidden_size = 64

# Define a list of optimizer configurations to try
optimizer_configs = [
    {"optimizer": torch.optim.SGD, "lr": 0.01},
    {"optimizer": torch.optim.SGD, "lr": 0.001},
    {"optimizer": torch.optim.SGD, "lr": 0.01, "momentum": 0.9},
    {"optimizer": torch.optim.Adam, "lr": 0.01},
    {"optimizer": torch.optim.Adam, "lr": 0.001},
    {"optimizer": torch.optim.RMSprop, "lr": 0.01},
    {"optimizer": torch.optim.RMSprop, "lr": 0.0001}
]

best_nn_model = None
best_nn_loss = float('inf')
best_optimizer_config = None

for config in optimizer_configs:
    # Create the neural network model with two hidden layers
    nn_model = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, 1)  # Single output neuron for continuous score
    )

    criterion = nn.MSELoss()

    # Initialize the optimizer with the current configuration
    optimizer = config["optimizer"](nn_model.parameters(), lr=config["lr"])

    if "momentum" in config:
        optimizer = torch.optim.SGD(nn_model.parameters(), lr=config["lr"], momentum=config["momentum"])

    num_epochs = 100
    loss = float('inf')
    for epoch in range(num_epochs):
        train_inputs = torch.tensor(X_train, dtype=torch.float32)
        train_labels = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)  # Continuous score

        # Forward pass
        train_outputs = nn_model(train_inputs)
        loss = criterion(train_outputs, train_labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training statistics (e.g., loss) as needed
        if (epoch + 1) % 10 == 0:
            print(f'Configuration: {config}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Check if this model has the lowest loss
    if loss < best_nn_loss:
        best_nn_loss = loss
        best_nn_model = nn_model
        best_optimizer_config = config
        
    # Evaluate the best NN model on the test set
    with torch.no_grad():
        test_inputs = torch.tensor(X_test, dtype=torch.float32)
        test_labels = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)
        test_outputs = nn_model(test_inputs)
        test_loss = criterion(test_outputs, test_labels)
    print("Test Loss:", test_loss.item())

# Print the best model's loss and keep it as "best_model"
print(f"Best Model Loss: {best_nn_loss}, Best Configuration: {best_optimizer_config}")

Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [10/100], Loss: 0.3845
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [20/100], Loss: 0.3821
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [30/100], Loss: 0.3804
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [40/100], Loss: 0.3790
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [50/100], Loss: 0.3778
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [60/100], Loss: 0.3769
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [70/100], Loss: 0.3760
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [80/100], Loss: 0.3753
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [90/100], Loss: 0.3747
Configuration: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.01}, Epoch [10

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Initialize variables to track the best model and its MSE
best_sk_model = None
best_sk_mse = float('inf')
best_sk_model_name = None 

models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "Elastic Net": ElasticNet(),
    "Bayesian Ridge": BayesianRidge(),
    "Extreme Gradient Boosting Regressor": XGBRegressor(objective="reg:squarederror", learning_rate=0.01, n_estimators=2000, max_depth=7)
}

for model_name, model in models.items():
    model.fit(X_train, y_train);
    y_preds = model.predict(X_test)

    # Evaluate the model performance 
    model_mse = mean_squared_error(y_preds, y_test)
    print(f"{model_name} MSE:", model_mse)

    # Check if this model has the lowest MSE
    if model_mse < best_sk_mse:
        best_sk_mse = model_mse
        best_sk_model = model
        best_sk_model_name = model_name

# Print the best model's MSE and store it as "best_model"
print(f"Best Model: {best_sk_model_name}, Best Model MSE: {best_sk_mse}")

Linear Regression MSE: 0.3597734920240081
Ridge MSE: 0.3597726009496486
Lasso MSE: 0.3735709224939708
Elastic Net MSE: 0.3735709224939708
Bayesian Ridge MSE: 0.35977255358364907
Extreme Gradient Boosting Regressor MSE: 0.3427200688512837
Best Model: Extreme Gradient Boosting Regressor, Best Model MSE: 0.3427200688512837


In [6]:
import pickle
import joblib

# Create and fit new model to all available data
final_model = XGBRegressor(objective="reg:squarederror", learning_rate=0.01, n_estimators=2000, max_depth=7)
final_model.fit(X, y)

# Save the best_sk_model to a file
model_filename = "agent_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(final_model, file)

scaler_filename = "scaler.pkl"
with open(scaler_filename, 'wb') as file:
    joblib.dump(scaler, file);